In [1]:
import os
from dotenv import load_dotenv
from rich import print

%load_ext rich
load_dotenv()


True

In [23]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import WebBaseLoader
from cleantext import clean
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_core.documents import Document

os.environ["USER_AGENT"] = (
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
)


In [11]:
def get_recipe_from_url(url: str) -> dict:
    website_data = []
    loader = WebBaseLoader(url)
    website_data.extend(loader.load())
    return website_data[0]


In [17]:
def sanitize_text(text: str) -> str:
    return clean(
        text,
        fix_unicode=True,
        to_ascii=True,
        lower=True,
        normalize_whitespace=True,
        no_line_breaks=True,
        strip_lines=True,
    )


In [18]:
TEST_URL = "https://www.recipetineats.com/honey-garlic-chicken/"

In [19]:
website_data = get_recipe_from_url(TEST_URL)


In [26]:
from typing import TypedDict, List


class Recipe(TypedDict):
    """Represents a simple recipe object."""

    title: str
    ingredients: List[str]
    instructions: List[str]

In [24]:
def format_recipe(website_data: Document) -> Recipe:
    model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    messages = [
        SystemMessage("Please extract recipes from the following website: "),
        HumanMessage(content=sanitize_text(website_data.page_content)),
    ]

    response = model.with_structured_output(Recipe).invoke(messages)
    return response

In [27]:
recipe = format_recipe(website_data)
print(recipe)

{
    'title': 'Honey Garlic Chicken Breast',
    'ingredients': [
        '500g / 1 lb chicken breast, boneless and skinless (2 pieces)',
        'Salt and pepper',
        '1/4 cup flour',
        '3 1/2 tbsp (50g) unsalted butter (or 2 1/2 tbsp olive oil)',
        '2 garlic cloves, minced',
        '1 1/2 tbsp apple cider vinegar (or white or other clear vinegar)',
        '1 tbsp soy sauce, light or all purpose',
        '1/3 cup honey (or maple syrup)'
    ],
    'instructions': [
        'Cut the breasts in half horizontally to create 4 steaks in total. Sprinkle each side with salt and 
pepper.',
        'Place flour in a shallow dish. Coat chicken in flour and shake off excess.',
        'Melt most of the butter in a large skillet over high heat, holding back about 1 tsp for later.',
        'Place chicken in skillet and cook for 2-3 minutes until golden. Turn and cook the other side for 1 
minute.',
        'Turn heat down slightly to medium high. Make a bit of room in the pan and add garlic and top with 
remaining dab of butter. Stir garlic briefly once butter melts.',
        'Add vinegar, soy sauce, and honey. Stir/shake pan to combine. Bring sauce to simmer, then simmer for 1 
minute or until slightly thickened.',
        'Turn chicken to coat in sauce. If the sauce gets too thick, add a touch of water and stir.',
        'Remove from stove immediately. Place chicken on plates and drizzle over remaining sauce.'
    ]
}